# Preprocessing Bike Sharing Data

In [1]:
import numpy as np
import pandas as pd
import datetime
from sqlalchemy import create_engine
import requests
import time

### Datasets

- Raw data: raw.csv

- Categorized in Trip and accessible: preprocessed.csv

- routed routed.csv

- pseudonomysed_raw.csv
    - pseudonomysed
    - trip ID, start time, end time, start location, stop location, duration, estimated distance<

- Cleaned according to plausability: cleaned.csv

- pseudonomysed.csv
    - cleaned and pseudonomysed
    - trip ID, start time, end time, start location, stop location, duration, estimated distance<

# Preprocessing

## get data from database

In [2]:
dbhost = '192.168.122.155'
dbname = 'bikes'
dbuser = 'bike_admin'
dbpassword = 'mybike'

In [3]:
engine = create_engine(f'postgresql://{dbuser}:{dbpassword}@{dbhost}:5432/{dbname}')

In [4]:
# Location changes of bikes
sql = """
select *
from (
    select  "bikeId", "timestamp", latitude, longitude,
			lag("bikeId") over (order by "bikeId" ASC, "timestamp" ASC) as prev_id,
            lead("bikeId") over (order by "bikeId" ASC, "timestamp" ASC) as next_id,
            lag("latitude") over (order by "bikeId" ASC, "timestamp" ASC) as prev_lat,
			lag("longitude") over (order by "bikeId" ASC, "timestamp" ASC) as prev_lon,
            lead("latitude") over (order by "bikeId" ASC, "timestamp" ASC) as next_lat,
            lead("longitude") over (order by "bikeId" ASC, "timestamp" ASC) as next_lon
    from public."bikeLocations"
    )x
	where ("bikeId" <> next_id) or
		   ("bikeId" <> prev_id) or
		   (latitude <> next_lat or longitude <> next_lon or latitude <> prev_lat or longitude <> prev_lon)
	
	--keep first and last instance of bike (bikeId <> prev_id or bikeId <> next_id)
	--keep all changes inbetween, where bike_Id is equal to next_id but lat or lon changed to prev or next
"""

df = pd.read_sql_query(sql,con=engine)

In [5]:
df.head()

,bikeId,timestamp,latitude,longitude,prev_id,next_id,prev_lat,prev_lon,next_lat,next_lon
0,04WkXwzVogAEe53iEG9WkZgBy1mxZLaN,2024-11-05 21:05:02.261832,50.588971,8.662469,04WkXwzVogAEe53iEG9WkZgBy1mxZLaN,04WkXwzVogAEe5kCEG9MkvgBy1mxZLaN,50.588971,8.662469,50.571893,8.690625
1,04WkXwzVogAEe5kCEG9MkvgBy1mxZLaN,2024-11-05 15:39:02.103801,50.571893,8.690625,04WkXwzVogAEe53iEG9WkZgBy1mxZLaN,04WkXwzVogAEe5kCEG9MkvgBy1mxZLaN,50.588971,8.662469,50.571800,8.690674
2,04WkXwzVogAEe5kCEG9MkvgBy1mxZLaN,2024-11-05 15:40:01.506829,50.571800,8.690674,04WkXwzVogAEe5kCEG9MkvgBy1mxZLaN,04WkXwzVogAEe5kCEG9MkvgBy1mxZLaN,50.571893,8.690625,50.571800,8.690674
3,04WkXwzVogAEe5kCEG9MkvgBy1mxZLaN,2024-11-05 15:41:01.897513,50.571800,8.690674,04WkXwzVogAEe5kCEG9MkvgBy1mxZLaN,04WkXwzVogAEeJ8sEG942AgBy1mxZLaN,50.571800,8.690674,50.587726,8.668395
4,04WkXwzVogAEeJ8sEG942AgBy1mxZLaN,2024-11-05 18:16:01.789876,50.587726,8.668395,04WkXwzVogAEe5kCEG9MkvgBy1mxZLaN,04WkXwzVogAEeJ8sEG942AgBy1mxZLaN,50.571800,8.690674,50.587726,8.668395


In [6]:
df.to_csv('../../data/raw.csv', index_label= 'id')

## preprocess

In [7]:
df = pd.read_csv('../../data/raw.csv', parse_dates=['timestamp'])

In [8]:
df.head()

,id,bikeId,timestamp,latitude,longitude,prev_id,next_id,prev_lat,prev_lon,next_lat,next_lon
0,0,04WkXwzVogAEe53iEG9WkZgBy1mxZLaN,2024-11-05 21:05:02.261832,50.588971,8.662469,04WkXwzVogAEe53iEG9WkZgBy1mxZLaN,04WkXwzVogAEe5kCEG9MkvgBy1mxZLaN,50.588971,8.662469,50.571893,8.690625
1,1,04WkXwzVogAEe5kCEG9MkvgBy1mxZLaN,2024-11-05 15:39:02.103801,50.571893,8.690625,04WkXwzVogAEe53iEG9WkZgBy1mxZLaN,04WkXwzVogAEe5kCEG9MkvgBy1mxZLaN,50.588971,8.662469,50.571800,8.690674
2,2,04WkXwzVogAEe5kCEG9MkvgBy1mxZLaN,2024-11-05 15:40:01.506829,50.571800,8.690674,04WkXwzVogAEe5kCEG9MkvgBy1mxZLaN,04WkXwzVogAEe5kCEG9MkvgBy1mxZLaN,50.571893,8.690625,50.571800,8.690674
3,3,04WkXwzVogAEe5kCEG9MkvgBy1mxZLaN,2024-11-05 15:41:01.897513,50.571800,8.690674,04WkXwzVogAEe5kCEG9MkvgBy1mxZLaN,04WkXwzVogAEeJ8sEG942AgBy1mxZLaN,50.571800,8.690674,50.587726,8.668395
4,4,04WkXwzVogAEeJ8sEG942AgBy1mxZLaN,2024-11-05 18:16:01.789876,50.587726,8.668395,04WkXwzVogAEe5kCEG9MkvgBy1mxZLaN,04WkXwzVogAEeJ8sEG942AgBy1mxZLaN,50.571800,8.690674,50.587726,8.668395


In [16]:
# correct wrong lat lons 
# get mode (trip or is bike accessible)

def preprocess(df):
    MODE_TRIP = 'trip'
    MODE_ACCESS = 'accessible'

    df['end_timestamp'] = None
    df['mode'] = None

    df.sort_values(by=['bikeId', 'timestamp'], inplace = True)
    df['end_timestamp'] = df.timestamp.shift(-1)
    
    df['next_lat'] = np.where(df.bikeId == df.bikeId.shift(-1), df.latitude.shift(-1), -1)
    df['next_lon'] = np.where(df.bikeId == df.bikeId.shift(-1), df.longitude.shift(-1), -1)

    # if the bike has moved its a trip, otherwise the bike was accessible at its location
    # round to 3 digits for lat lon (trip should be longer than approx. 100 meters)
    df['mode'] = np.where((df.latitude.round(3) != df.next_lat.round(3)) | (df.longitude.round(4) != df.next_lon.round(4)), MODE_TRIP, MODE_ACCESS)
    
    # if bike Id is not the same as next, then this is the final time stamp and the next_timestamp is set to None
    df['end_timestamp'] = df.end_timestamp.where(df.bikeId == df.bikeId.shift(-1),None, axis=0)
    
    # delete last instances of all bikes
    df = df[df.next_lat.notnull()]
    
    return df

In [17]:
df = preprocess(df)

In [18]:
df

,id,bikeId,timestamp,latitude,longitude,prev_id,next_id,prev_lat,prev_lon,next_lat,next_lon,end_timestamp,mode
0,0,04WkXwzVogAEe53iEG9WkZgBy1mxZLaN,2024-11-05 20:51:02.001591,50.588971,8.662469,04WkXwzVogAEe53iEG9WkZgBy1mxZLaN,04WkXwzVogAEe5kCEG9MkvgBy1mxZLaN,50.588971,8.662469,-1.000000,-1.000000,NaT,trip
1,1,04WkXwzVogAEe5kCEG9MkvgBy1mxZLaN,2024-11-05 15:39:02.103801,50.571893,8.690625,04WkXwzVogAEe53iEG9WkZgBy1mxZLaN,04WkXwzVogAEe5kCEG9MkvgBy1mxZLaN,50.588971,8.662469,50.571800,8.690674,2024-11-05 15:40:01.506829,trip
2,2,04WkXwzVogAEe5kCEG9MkvgBy1mxZLaN,2024-11-05 15:40:01.506829,50.571800,8.690674,04WkXwzVogAEe5kCEG9MkvgBy1mxZLaN,04WkXwzVogAEe5kCEG9MkvgBy1mxZLaN,50.571893,8.690625,50.571800,8.690674,2024-11-05 15:41:01.897513,accessible
3,3,04WkXwzVogAEe5kCEG9MkvgBy1mxZLaN,2024-11-05 15:41:01.897513,50.571800,8.690674,04WkXwzVogAEe5kCEG9MkvgBy1mxZLaN,04WkXwzVogAEeJ8sEG942AgBy1mxZLaN,50.571800,8.690674,-1.000000,-1.000000,NaT,trip
4,4,04WkXwzVogAEeJ8sEG942AgBy1mxZLaN,2024-11-05 18:16:01.789876,50.587726,8.668395,04WkXwzVogAEe5kCEG9MkvgBy1mxZLaN,04WkXwzVogAEeJ8sEG942AgBy1mxZLaN,50.571800,8.690674,50.587726,8.668395,2024-11-05 20:51:02.001591,accessible
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1426,1426,zBbYZ49oNn41zWKiwd2q2l6K0pdmPy71,2024-11-05 18:02:01.874255,50.590141,8.684306,zBbYZ49oNn41zPXSwd2xor6K0pdmPy71,zBbYZ49oNn41zWKiwd2q2l6K0pdmPy71,50.587726,8.668395,50.590141,8.684306,2024-11-05 20:51:02.001591,accessible
1427,1427,zBbYZ49oNn41zWKiwd2q2l6K0pdmPy71,2024-11-05 20:51:02.001591,50.590141,8.684306,zBbYZ49oNn41zWKiwd2q2l6K0pdmPy71,zBbYZ49oNn41zWVTwd2yrX6K0pdmPy71,50.590141,8.684306,-1.000000,-1.000000,NaT,trip
1428,1428,zBbYZ49oNn41zWVTwd2yrX6K0pdmPy71,2024-11-05 15:25:58.716826,50.585584,8.684702,zBbYZ49oNn41zWKiwd2q2l6K0pdmPy71,zBbYZ49oNn41zWVTwd2yrX6K0pdmPy71,50.590141,8.684306,50.585584,8.684702,2024-11-05 16:27:02.199705,accessible
1429,1429,zBbYZ49oNn41zWVTwd2yrX6K0pdmPy71,2024-11-05 16:27:02.199705,50.585584,8.684702,zBbYZ49oNn41zWVTwd2yrX6K0pdmPy71,zBbYZ49oNn41zWwIwd29ZM6K0pdmPy71,50.585584,8.684702,-1.000000,-1.000000,NaT,trip


In [20]:
df.to_csv('../../data/preprocessed.csv', index_label= 'id')

## Routing

In [4]:
def routing(start_lon, start_lat, end_lon, end_lat):
    start = str(start_lon) + ',' + str(start_lat)
    end = str(end_lon) + ',' + str(end_lat)
    params = {'start':start,
              'end':end,
              'modal':'bicycle'
             }
    r = requests.get('http://localhost:1717/route', params = params)
    response = r.json()
    return response

In [181]:
df = pd.DataFrame()
for chunk in pd.read_csv('../../data/preprocessed.csv', index_col='id', parse_dates=['timestamp', 'end_timestamp'],chunksize=100000):
    start = time.time()
    chunk['response'] = None
    chunk['geometry'] = None
    chunk['distance'] = None
    chunk['calc_time'] = None
    chunk['waypoints'] = None
    
    # only do routing for trips
    selection = (chunk['mode']=='trip') & (chunk.next_lat.notnull())
    chunk.loc[selection, 'response'] = chunk.loc[selection].apply(lambda row: routing(row.longitude, row.latitude, row.next_lon, row.next_lat), axis=1)
    chunk.loc[selection, 'geometry'] = chunk.loc[selection]['response'].apply(lambda row: row['routes'][0]['geometry'])
    chunk.loc[selection, 'distance'] = chunk.loc[selection]['response'].apply(lambda row: row['routes'][0]['distance'])
    chunk.loc[selection, 'calc_time'] = chunk.loc[selection]['response'].apply(lambda row: row['routes'][0]['duration'])
    chunk.loc[selection, 'waypoints'] = chunk.loc[selection]['response'].apply(lambda row: row['waypoints'])
    end = time.time()
    print(end - start)
    df = df.append(chunk)

214.57879996299744
223.48249912261963
214.69404888153076
189.998055934906
177.86353087425232
178.07002592086792
178.5570878982544
490.3402931690216
514.6639800071716
230.62421798706055
204.9859230518341
166.9845311641693
173.87214183807373
163.3482518196106
156.35346794128418
154.9044930934906
168.71858096122742
179.56365704536438
154.8981430530548
153.71806001663208
161.40800881385803
160.61402201652527
484.7227940559387
165.4938759803772
144.62416791915894
160.95510387420654
188.71538925170898
160.1147141456604
156.14063096046448
145.88051581382751
151.20413494110107
5570.233852863312
168.08731317520142
156.1642780303955
168.08376622200012
180.47953414916992
164.22090196609497
177.3228931427002
9975.520352125168
155.88235807418823
154.1496877670288
146.9719421863556
1111.4445898532867
161.26306581497192
165.64914107322693
157.92316102981567
171.8003692626953
163.10643601417542
150.91775488853455
175.11917686462402
0.3518679141998291


In [182]:
# df.to_csv('../../data/routed.csv')

## Pseudonomysation of not cleaned data

In [5]:
df = pd.read_csv('../../data/routed.csv', parse_dates=['timestamp','end_timestamp'])

In [40]:
# pseudonomoysed
pseudon = df.copy(deep=True)
pseudon['starttime'] = pseudon.timestamp.apply(lambda dt: datetime.datetime(dt.year, dt.month, dt.day, dt.hour,15*(dt.minute // 15)))
pseudon['endtime'] = pseudon.end_timestamp.apply(lambda dt: datetime.datetime(dt.year, dt.month, dt.day, dt.hour,15*(dt.minute // 15)) if isinstance(dt, pd.Timestamp) else None)
pseudon.reset_index()
pseudon.latitude = pseudon.latitude.round(3)
pseudon.longitude = pseudon.longitude.round(3)
pseudon.next_lat = pseudon.next_lat.round(3)
pseudon.next_lon = pseudon.next_lon.round(3)
pseudon = pseudon[pseudon['mode'] == 'trip'].drop(['providerId', 'bikeId', 'timestamp', 'end_timestamp', 'next_id','prev_id', 'mode', 'prev_lat', 'prev_lon', 'response', 'calc_time','geometry', 'waypoints'], axis=1).copy(deep=True)

In [41]:
pseudon.to_csv('../../data/pseudonomysed_raw.csv', index_label='id')

## Data Cleaning

In [15]:
df = pd.read_csv('../../data/routed.csv', parse_dates=['timestamp','end_timestamp'])

In [16]:
def cleaning(df):
    df['duration'] = df.end_timestamp - df.timestamp
    df['speed'] = np.where(df.distance.notnull(), ((df.distance/1000) / (df.duration.apply(lambda x: x.total_seconds()/3600))), None)
 
    # delete instances with unplausible locations (outside of Berlin)
    df.drop(df[df.longitude > 13.8].index, inplace=True)
    df.drop(df[df.next_lon > 13.8].index, inplace=True)

    df.drop(df[df.longitude < 13.0].index, inplace=True)
    df.drop(df[df.next_lon < 13.0].index, inplace=True)

    df.drop(df[df.latitude > 52.7].index, inplace=True)
    df.drop(df[df.next_lat > 52.7].index, inplace=True)

    df.drop(df[df.latitude < 52.3].index, inplace=True)
    df.drop(df[df.next_lat < 52.3].index, inplace=True)

    
    # Check for unplausible data
    
    # drop trips that are longer than 24 hours
    df = df.drop(df[(df['mode'] == 'trip') & (df.duration > datetime.timedelta(hours=24))].index)

    # drop trips before 1.4.2019 
    df = df.drop(df[df['timestamp'] <  pd.Timestamp(2019,4,1)].index)
    
    # drop trips that are smaller than 200 m or faster than 30 km/h
    df = df[(df.distance > 200) & (df.speed < 30)]
    
    # delete all trips starting at 4am
    df = df[df.timestamp.apply(lambda x: x.time().hour) != 4]
    
    # drop all last occurencies
    ids = df.sort_values(by=['bikeId', 'timestamp']).groupby('bikeId').last()['id.1']
    df = df.drop(ids.tolist())
    
    return df

In [17]:
df = cleaning(df)

In [18]:
# drop all last occurencies
#ids = df.sort_values(by=['bikeId', 'timestamp']).groupby('bikeId').last()['id.1']
#df = df.drop(ids.tolist())

In [19]:
df.drop('id.1', axis=1, inplace=True)

In [20]:
df.shape

(530026, 21)

In [22]:
df.to_csv('../../data/cleaned.csv', index_label = 'id')

## Pseudonymisation of cleaned data

https://datasmart.ash.harvard.edu/news/article/civic-analytics-network-dockless-mobility-open-letter

https://data.louisvilleky.gov/dataset/dockless-vehicles

In [3]:
# df = pd.read_csv('../../data/cleaned.csv', parse_dates=['timestamp','end_timestamp'])

In [5]:
# pseudonomoysed cleaned without Mobike
pseudon = df[df.providerId != 2].copy(deep=True)

pseudon['starttime'] = pseudon.timestamp.apply(lambda dt: datetime.datetime(dt.year, dt.month, dt.day, dt.hour,15*(dt.minute // 15)))
pseudon['endtime'] = pseudon.end_timestamp.apply(lambda dt: datetime.datetime(dt.year, dt.month, dt.day, dt.hour,15*(dt.minute // 15)))
# pseudon.end_timestamp.apply(lambda dt: datetime.datetime(dt.year, dt.month, dt.day, dt.hour,15*(dt.minute // 15)) if isinstance(dt, pd.Timestamp) else None)
pseudon.reset_index()
pseudon.latitude = pseudon.latitude.round(3)
pseudon.longitude = pseudon.longitude.round(3)
pseudon.next_lat = pseudon.next_lat.round(3)
pseudon.next_lon = pseudon.next_lon.round(3)
pseudon = pseudon[pseudon['mode'] == 'trip'].drop(['providerId', 'bikeId', 'timestamp', 'end_timestamp', 'next_id','prev_id', 'mode', 'prev_lat', 'prev_lon', 'response', 'calc_time','geometry', 'waypoints'], axis=1).copy(deep=True)
pseudon.rename(columns={'latitude':'start_lat', 'longitude':'start_lng', 'next_lat': 'end_lat', 'next_lon': 'end_lng'}, inplace=True)

In [8]:
pseudon.drop('id.1', axis=1, inplace=True)

In [9]:
pseudon.to_csv('../../data/pseudonomysed.csv', index_label='id')